In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

train_data_m = pd.read_csv("https://raw.githubusercontent.com/mohammed1916/ml/main/Datasets/JobOffer.csv")
test_data_m = pd.read_csv("https://raw.githubusercontent.com/mohammed1916/ml/main/Datasets/JobOfferTest.csv")
print(train_data_m)
column_values = {
    'CGPA': ['<8', '>=8', '>=9'],
    'Interactive': ['Yes', 'No'],
    'Practical Knowledge': ['Average', 'Good', 'Very Good'],
    'Communication Skills': ['Poor', 'Moderate', 'Good'],
    'Job Offer': ['Yes', 'No']
}
gini_index = lambda p,q: 1 - np.power(p,2) - np.power(q,2)
# gini_index((7/8),(1/8))
# Output: 0.21875

column_names = train_data_m.columns.tolist()
# seperate the list into dependent and independent

# Define dependent and independent variable lists
dependent_variables = ['Job Offer']
independent_variables = [col for col in column_names if col not in dependent_variables]

# Print the lists
print("Dependent variables:", dependent_variables)
print("Independent variables:", independent_variables)
target = dependent_variables

# initialize an empty dataframe
class info:
  def __init__(self):
    self.gini_index_total_list = pd.DataFrame(columns=['Subsets', 'GiniIndex'])
    self.curr_gini_attr_list = pd.DataFrame(columns=['Attribute','Δ Gini'])
    self.best_min_split_sets = pd.DataFrame(columns=['Attribute','Set'])
    self.independent_variables = independent_variables
    self.train_data_m = train_data_m
    self.shouldRun = True
    self.tree = {}

  def add_to_tree(self, attribute, split_set, left_data, right_data):
    # if attribute in self.tree:
    #   self.tree[attribute] = self.tree[attribute] + {
    #     'split_set': split_set,
    #     'left': left_data,
    #     'right': right_data
    #   }
    # else:
    self.tree[attribute] = {
      'split_set': split_set,
      'left': left_data,
      'right': right_data
    }
info_o = info()

  CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good                 Good       Yes
1  >=8          No                Good             Moderate       Yes
2  >=9          No             Average                 Poor        No
3   <8          No             Average                 Good        No
4  >=8         Yes                Good             Moderate       Yes
5  >=9         Yes                Good             Moderate       Yes
6   <8         Yes                Good                 Poor        No
7  >=9          No           Very Good                 Good       Yes
8  >=8         Yes                Good                 Good       Yes
9  >=8         Yes             Average                 Good       Yes
Dependent variables: ['Job Offer']
Independent variables: ['CGPA', 'Interactive', 'Practical Knowledge', 'Communication Skills']


In [ ]:
def run(curr_attr):
  unique_categories = info_o.train_data_m[curr_attr].unique()
  print("unique_categories",unique_categories)

  subset_combinations = []
  length = 2 if len(unique_categories) > 2 else 1
  # Generate combinations of length 1 to len(unique_categories) - 1
  for r in range(length, len(unique_categories)):
      # Generate combinations of length r
      for comb in combinations(unique_categories, r):
          # Add the combination along with the remaining categories as a tuple to the subset_combinations list
          print("comb:",comb)
          subset_combinations.append((comb, tuple(set(unique_categories) - set(comb))))

  # Sort the subset_combinations list for consistency
  subset_combinations.sort()

  # Convert the list of tuples to a set of frozensets to remove duplicates
  subset_combinations = {frozenset(sub) for sub in subset_combinations}

  # Convert the set of frozensets back to a list of tuples for consistency
  subset_combinations = [tuple(sub) for sub in subset_combinations]

  # Print the result
  print("subset_combinations: ",subset_combinations)

  def calc_count_probability_target(current_class):
    target_class = 'Job Offer'
    positive_count = current_class[info_o.train_data_m[target_class] == 'Yes'].shape[0]
    negative_count = current_class[info_o.train_data_m[target_class] == 'No'].shape[0]
    total_count = positive_count + negative_count


    positive_proportion = positive_count / total_count
    negative_proportion = negative_count / total_count

    gini = gini_index(positive_proportion, negative_proportion)
    return gini
  curr_target_gini = calc_count_probability_target(info_o.train_data_m)
  print(f"Gini index for Target: {curr_target_gini:.4f}")
  def get_count(category, column):
    print("category, column",category, column)
    subset = info_o.train_data_m[info_o.train_data_m[column] == category]
    print("subset:",subset)
    positive_count = subset[subset['Job Offer'] == 'Yes'].shape[0]
    negative_count = subset[subset['Job Offer'] == 'No'].shape[0]
    return [positive_count, negative_count]
  def gen_table(unique_categories, column):
      curr_class = pd.DataFrame(columns=['Category', 'Count [Y,N]'])
      for category in unique_categories:
          count = get_count(category, column)  # Assuming get_count() returns the count for the category
          curr_class = curr_class.append({'Category': category, 'Count [Y,N]': count}, ignore_index=True)
      return curr_class

  curr_df = gen_table(unique_categories, curr_attr)
  print("curr_df",curr_df)

  def calc_count_probability_0(p_n,category_tuple):
    # positive_count, negative_count = positive_count + p_n[0], negative_count+ p_n[1]
    p_n= str(p_n[0])
    print("p_n:",p_n)
    positive_count, negative_count = curr_df.loc[curr_df['Category'] == p_n, 'Count [Y,N]'].values[0]
    total_count = positive_count + negative_count
    print("positive_count, negative_count",positive_count, negative_count)

    positive_proportion = positive_count / total_count
    negative_proportion = negative_count / total_count

    gini = gini_index(positive_proportion, negative_proportion)
    return gini, total_count

  def calc_count_probability(p_n,category_tuple):
    # positive_count, negative_count = positive_count + p_n[0], negative_count+ p_n[1]
    print("p_n:",p_n)
    positive_count, negative_count = get_count(p_n, category_tuple)
    total_count = positive_count + negative_count
    print("positive_count, negative_count",positive_count, negative_count)

    positive_proportion = positive_count / total_count
    negative_proportion = negative_count / total_count

    gini = gini_index(positive_proportion, negative_proportion)
    return gini, total_count
  def calc_count_probability_2(p_n1,p_n2,category_tuple):
    res1 = get_count(p_n1, category_tuple)
    res2 = get_count(p_n2, category_tuple)
    positive_count, negative_count = res1[0] + res2[0], res1[1] + res2[1]
    total_count = positive_count + negative_count


    positive_proportion = positive_count / total_count
    negative_proportion = negative_count / total_count

    gini = gini_index(positive_proportion, negative_proportion)
    return gini,total_count
  # curr_df.loc[curr_df['Category'] == '>=9', 'Count [Y,N]'].values[0]
  subset_combinations = [tuple(sorted(pair, key=lambda x: len(x), reverse=True)) for pair in subset_combinations]
  # for s in subset_combinations:
  #   print(s)
  # for s in subset_combinations:
  #   print([s[0][0],s[0][1]])
  # get_count(s[1][0], curr_attr)


  for s in subset_combinations:
    if len(s[0]) > 1:
      set1, total_set1 = calc_count_probability_2(s[0][0],s[0][1],curr_attr)
      print(f"Gini index for {s[0][0]},{s[0][1]}: {set1:.4f}")
      set2, total_set2 = calc_count_probability(s[1][0],curr_attr)
      print(f"Gini index for {s[1][0]}: {set2:.4f}")
    else:
      print("s[0]:",s[0])
      print("s[1]:",s[1])
      set1, total_set1 = calc_count_probability_0(s[0],curr_attr)
      print(f"Gini index for {s[0]}: {set1:.4f}")
      set2, total_set2 = calc_count_probability_0(s[1],curr_attr)
      print(f"Gini index for {s[1]}: {set2:.4f}")

    total = total_set1 + total_set2
    weighted_gini = (total_set1/total) * set1 + (total_set2/total)  * set2
    print(f"Weighted Gini index for {s}: {weighted_gini:.4f}")
    info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)

  print(info_o.gini_index_total_list)

  # find minimum of GiniIndex in info_o.gini_index_total_list

  min_gini_index = info_o.gini_index_total_list['GiniIndex'].min()
  min_gini_row = info_o.gini_index_total_list[info_o.gini_index_total_list['GiniIndex'] == min_gini_index]
  print(f"Minimum Gini Index: {min_gini_index}")
  print(f"Corresponding Subsets: {min_gini_row['Subsets'].values[0]}")
  info_o.best_min_split_sets = info_o.best_min_split_sets.append({'Attribute': curr_attr, 'Set': min_gini_row['Subsets'].values[0]}, ignore_index=True)

  res = curr_target_gini - min_gini_index
  info_o.curr_gini_attr_list = info_o.curr_gini_attr_list.append({'Attribute':curr_attr,'Δ Gini':res}, ignore_index=True)
  print(info_o.curr_gini_attr_list)


  # Clear for next run:
  info_o.gini_index_total_list = pd.DataFrame(columns=['Subsets', 'GiniIndex'])


In [ ]:
def run_outer():
  for attr in info_o.independent_variables:
    print("*"*10,attr,"*"*10)
    run(attr)

  max_gini_row = info_o.curr_gini_attr_list['Δ Gini'].idxmax()
  max_gini_attr = info_o.curr_gini_attr_list.loc[max_gini_row, 'Attribute']
  print(f"Attribute with maximum Δ Gini: {max_gini_attr}")


  best_split_set = info_o.best_min_split_sets[info_o.best_min_split_sets['Attribute'] == max_gini_attr]['Set'].values[0]
  print(f"Best split set for {max_gini_attr}: {best_split_set}")

  # Create two dataframes based on the best split set
  train_data_m_left = info_o.train_data_m[info_o.train_data_m[max_gini_attr].isin(best_split_set[0])]
  train_data_m_right = info_o.train_data_m[info_o.train_data_m[max_gini_attr].isin(best_split_set[1])]

  # Print the resulting dataframes
  print("Left DataFrame:")
  print(train_data_m_left)
  print("Right DataFrame:")
  print(train_data_m_right)


  def check_leaf(df):
    # Check if all values in "Job Offer" column are the same
    unique_values = df["Job Offer"].unique()
    if len(unique_values) == 1:
      return True
    else:
      return False

  is_leaf_right = check_leaf(train_data_m_right)
  is_leaf_left = check_leaf(train_data_m_left)

  print(f"Train data right is a leaf node: {is_leaf_right}")
  print(f"Train data left is a leaf node: {is_leaf_left}")

  if is_leaf_right and is_leaf_left:
    info_o.shouldRun = False

  def modify_independent_variables(independent_variables, max_gini_attr):
    # Remove the attribute with the best split from the independent variables list
    print("independent_variables: ", independent_variables)


    independent_variables.remove(max_gini_attr)

    # Return the modified list
    return independent_variables

  if info_o.shouldRun:
    if not is_leaf_left:
      info_o.independent_variables = modify_independent_variables(info_o.independent_variables, max_gini_attr)

    elif not is_leaf_right:
      info_o.independent_variables = modify_independent_variables(info_o.independent_variables, max_gini_attr)

    if len(independent_variables) < 2:
      info_o.shouldRun = False
    print("info_o.shouldRun",info_o.shouldRun)
    print("Added to tree:",max_gini_attr)
    if info_o == None:
      info_o.add_to_tree(max_gini_attr, best_split_set, train_data_m_left, train_data_m_right)
    else:
      info_o.add_to_tree(max_gini_attr, best_split_set, train_data_m_left, train_data_m_right)
    train_data_m = train_data_m_left if not is_leaf_left else train_data_m_right

  # for attr in info_o.independent_variables:
  #   print("*"*10,attr,"*"*10)
  #   run(attr,)

In [ ]:
i=0
while info_o.shouldRun:
  info_o.curr_gini_attr_list = pd.DataFrame(columns=['Attribute','Δ Gini'])
  print("*-#"*10,i,"*"*10)
  run_outer()
  i+=1

*-#*-#*-#*-#*-#*-#*-#*-#*-#*-# 0 **********
********** CGPA **********
unique_categories ['>=9' '>=8' '<8']
comb: ('>=9', '>=8')
comb: ('>=9', '<8')
comb: ('>=8', '<8')
subset_combinations:  [(('>=9',), ('>=8', '<8')), (('<8',), ('>=9', '>=8')), (('>=9', '<8'), ('>=8',))]
Gini index for Target: 0.4200
category, column >=9 CGPA
subset:   CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good                 Good       Yes
2  >=9          No             Average                 Poor        No
5  >=9         Yes                Good             Moderate       Yes
7  >=9          No           Very Good                 Good       Yes
category, column >=8 CGPA
subset:   CGPA Interactive Practical Knowledge Communication Skills Job Offer
1  >=8          No                Good             Moderate       Yes
4  >=8         Yes                Good             Moderate       Yes
8  >=8         Yes                Good                 Good       Yes

<ipython-input-158-235bb7334f55>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_class = curr_class.append({'Category': category, 'Count [Y,N]': count}, ignore_index=True)
<ipython-input-158-235bb7334f55>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_class = curr_class.append({'Category': category, 'Count [Y,N]': count}, ignore_index=True)
<ipython-input-158-235bb7334f55>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_class = curr_class.append({'Category': category, 'Count [Y,N]': count}, ignore_index=True)
<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total

   CGPA Interactive Practical Knowledge Communication Skills Job Offer
1  >=8          No                Good             Moderate       Yes
4  >=8         Yes                Good             Moderate       Yes
5  >=9         Yes                Good             Moderate       Yes
6   <8         Yes                Good                 Poor        No
8  >=8         Yes                Good                 Good       Yes
Gini index for Very Good,Good: 0.2449
p_n: Average
category, column Average Practical Knowledge
subset:   CGPA Interactive Practical Knowledge Communication Skills Job Offer
2  >=9          No             Average                 Poor        No
3   <8          No             Average                 Good        No
9  >=8         Yes             Average                 Good       Yes
positive_count, negative_count 1 2
Gini index for Average: 0.4444
Weighted Gini index for (('Very Good', 'Good'), ('Average',)): 0.3048
                           Subsets  GiniIndex
0  ((Very Goo

<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.best_min_split_sets = info_o.best_min_split_sets.append({'Attribute': curr_attr, 'Set': min_gini_row['Subsets'].values[0]}, ignore_index=True)
<ipython-input-158-235bb7334f55>:135: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.curr_gini_attr_list = info_o.curr_gini_attr_list.append({'Attribute':curr_attr,'Δ Gini':res}, ignore_index=True)
<ipython-input-158-235bb7334f55>:52: FutureWarning: The frame.append method is de


Added to tree: CGPA
Train data right is a leaf node: True
Train data left is a leaf node: False
independent_variables:  ['CGPA', 'Interactive', 'Practical Knowledge', 'Communication Skills']
info_o.shouldRun True
*-#*-#*-#*-#*-#*-#*-#*-#*-#*-# 1 **********
********** Interactive **********
unique_categories ['Yes' 'No']
comb: ('Yes',)
comb: ('No',)
subset_combinations:  [(('Yes',), ('No',))]
Gini index for Target: 0.4200
category, column Yes Interactive
subset:   CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good                 Good       Yes
4  >=8         Yes                Good             Moderate       Yes
5  >=9         Yes                Good             Moderate       Yes
6   <8         Yes                Good                 Poor        No
8  >=8         Yes                Good                 Good       Yes
9  >=8         Yes             Average                 Good       Yes
category, column No Interactive
subset:   C

...

<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.best_min_split_sets = info_o.best_min_split_sets.append({'Attribute': curr_attr, 'Set': min_gini_row['Subsets'].values[0]}, ignore_index=True)
<ipython-input-158-235bb7334f55>:135: FutureWarning: The frame.append m

                           Subsets  GiniIndex
0  ((Very Good, Average), (Good,))   0.400000
1  ((Good, Average), (Very Good,))   0.375000
2  ((Very Good, Good), (Average,))   0.304762
Minimum Gini Index: 0.3047619047619048
Corresponding Subsets: (('Very Good', 'Good'), ('Average',))
             Attribute    Δ Gini
0          Interactive  0.053333
1  Practical Knowledge  0.115238
********** Communication Skills **********
unique_categories ['Good' 'Moderate' 'Poor']
comb: ('Good', 'Moderate')
comb: ('Good', 'Poor')
comb: ('Moderate', 'Poor')
subset_combinations:  [(('Good', 'Poor'), ('Moderate',)), (('Good',), ('Moderate', 'Poor')), (('Poor',), ('Good', 'Moderate'))]
Gini index for Target: 0.4200
category, column Good Communication Skills
subset:   CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good                 Good       Yes
3   <8          No             Average                 Good        No
7  >=9          No           Very

<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:132: FutureWarning: The frame.append method is deprecated 

   CGPA Interactive Practical Knowledge Communication Skills Job Offer
2  >=9          No             Average                 Poor        No
6   <8         Yes                Good                 Poor        No
positive_count, negative_count 0 2
Gini index for Poor: 0.0000
Weighted Gini index for (('Good', 'Moderate'), ('Poor',)): 0.1750
                       Subsets  GiniIndex
0  ((Good, Poor), (Moderate,))   0.342857
1  ((Moderate, Poor), (Good,))   0.400000
2  ((Good, Moderate), (Poor,))   0.175000
Minimum Gini Index: 0.17500000000000002
Corresponding Subsets: (('Good', 'Moderate'), ('Poor',))
              Attribute    Δ Gini
0           Interactive  0.053333
1   Practical Knowledge  0.115238
2  Communication Skills  0.245000
Attribute with maximum Δ Gini: Communication Skills
Best split set for Communication Skills: (('Good', 'Moderate'), ('Poor',))
Left DataFrame:
  CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good        

<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info_o.gini_index_total_list = info_o.gini_index_total_list.append({'Subsets': s, 'GiniIndex': weighted_gini}, ignore_index=True)
<ipython-input-158-235bb7334f55>:132: FutureWarning: The frame.append method is deprecated 

In [ ]:
import json

def print_tree_pretty(tree, indent="        "):
  """
  Prints a decision tree in a pretty format.

  Args:
    tree: A dictionary representing the decision tree.
    indent: The indentation to use for the current level of the tree.
  """
  for attribute, subtree in tree.items():
    print(f"{indent}{attribute}")
    if isinstance(subtree, dict):
      print_tree_pretty(subtree, indent + "  ")
    # else:
      # print(f"{indent}  {subtree}")

print("Decision Tree:")
print_tree_pretty(info_o.tree)

Decision Tree:
        CGPA
          split_set
          left
          right
        Communication Skills
          split_set
          left
          right
        Practical Knowledge
          split_set
          left
          right


In [ ]:
# print info_o.tree in pretty

import json

def print_tree_pretty(tree, indent=""):
  for attribute, subtree in tree.items():
    print(f"{indent}{attribute}")
    if isinstance(subtree, dict):
      print_tree_pretty(subtree, indent + "  ")
    else:
      print(f"{indent}  {subtree}")

print("Decision Tree:")
print_tree_pretty(info_o.tree)


Decision Tree:
CGPA
  split_set
    (('>=9', '>=8'), ('<8',))
  left
      CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good                 Good       Yes
1  >=8          No                Good             Moderate       Yes
2  >=9          No             Average                 Poor        No
4  >=8         Yes                Good             Moderate       Yes
5  >=9         Yes                Good             Moderate       Yes
7  >=9          No           Very Good                 Good       Yes
8  >=8         Yes                Good                 Good       Yes
9  >=8         Yes             Average                 Good       Yes
  right
      CGPA Interactive Practical Knowledge Communication Skills Job Offer
3   <8          No             Average                 Good        No
6   <8         Yes                Good                 Poor        No
Communication Skills
  split_set
    (('Good', 'Moderate'), ('Poor',))
  le

In [ ]:
print(info_o.tree)

{'CGPA': {'split_set': (('>=9', '>=8'), ('<8',)), 'left':   CGPA Interactive Practical Knowledge Communication Skills Job Offer
0  >=9         Yes           Very Good                 Good       Yes
1  >=8          No                Good             Moderate       Yes
2  >=9          No             Average                 Poor        No
4  >=8         Yes                Good             Moderate       Yes
5  >=9         Yes                Good             Moderate       Yes
7  >=9          No           Very Good                 Good       Yes
8  >=8         Yes                Good                 Good       Yes
9  >=8         Yes             Average                 Good       Yes, 'right':   CGPA Interactive Practical Knowledge Communication Skills Job Offer
3   <8          No             Average                 Good        No
6   <8         Yes                Good                 Poor        No}, 'Communication Skills': {'split_set': (('Good', 'Moderate'), ('Poor',)), 'left':   CGPA In